In [3]:
import sys  
sys.path.insert(0, '..')

from config import Config

import pandas as pd
from sqlalchemy import create_engine

db = create_engine(Config.SQLALCHEMY_DATABASE_URI)

In [26]:
odhad = pd.read_sql_query(
    """
    select vekova_skupina, min(datum) datum, min(pocet) pocet from (
        select min(vekova_skupina) vekova_skupina, sum(pocet) * 0.7 pocet
        from populace_kategorie
        join populace on vek >= min_vek and orp_kod = 'CZ0'
        where min_vek >= 18
        group by vekova_skupina
    ) t
    join (
        select datum, sum(pocet) over (order by datum rows between unbounded preceding and current row) as celkem_lidi 
        from (
            select datum, sum(pocet / davky) as pocet 
            from dodavky_vakcin d 
            join vakciny v on (d.vyrobce = v.vyrobce)
            group by datum
        ) t1
    ) t2
    on pocet < celkem_lidi
    group by vekova_skupina
    order by vekova_skupina, datum
    """,
    db
)
odhad

,vekova_skupina,datum,pocet
0,18-24,2021-07-01,6086131.8
1,25-29,2021-07-01,5615872.5
2,30-34,2021-07-01,5159548.1
3,35-39,2021-06-01,4654059.2
4,40-44,2021-06-01,4119960.6
5,45-49,2021-06-01,3473548.4
6,50-54,2021-05-01,2886053.8
7,55-59,2021-05-01,2405688.6
8,60-64,2021-05-01,1948237.9
9,65-69,2021-04-01,1492141.0
